In [16]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import json
import time
import re

# --- SETTINGS ---
# Установите True, чтобы видеть подробную отладочную информацию
DEBUG = True
# --- /SETTINGS ---

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

BASE_URL = "https://finuslugi.ru"

def get_fund_links(list_url):
    """
    Получает все ссылки на карточки фондов со страницы списка.
    """
    try:
        print(f"\n[INFO] Загружаю список фондов: {list_url}")
        response = requests.get(list_url, headers=HEADERS)
        print(f"[DEBUG] Статус-код ответа: {response.status_code}")
        response.raise_for_status()
        
        if DEBUG:
            print(f"[DEBUG] Получен HTML (первые 300 символов): {response.text[:300]}...")

        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Ищем ссылки только в основном контенте страницы, чтобы избежать лишних
        main_content = soup.find('main')
        if not main_content:
             print(f"[WARNING] Не найден тег <main> на странице: {list_url}")
             main_content = soup # Fallback to searching the whole page

        fund_links = main_content.find_all('a', href=re.compile(r'/invest/funds/[\w-]+'))
        
        if not fund_links:
            print(f"[WARNING] Не найдено ссылок на фонды на странице: {list_url}")
            return []
            
        extracted_urls = {BASE_URL + link.get('href') for link in fund_links}
        
        print(f"[INFO] Найдено {len(extracted_urls)} уникальных ссылок на фонды.")
        return list(extracted_urls)

    except requests.exceptions.RequestException as e:
        print(f"[ERROR] Ошибка при загрузке страницы списка {list_url}: {e}")
        return []
    
def parse_fund_card(card_url):
    """
    Извлекает данные из JSON, встроенного в HTML страницы.
    """
    try:
        print(f"  > Парсинг карточки: {card_url}")
        response = requests.get(card_url, headers=HEADERS)
        print(f"  [DEBUG] Статус-код ответа: {response.status_code}")
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        print(response.text)
        
        # Ищем тег script, содержащий JSON. re.DOTALL позволяет точке (.) находить и переносы строк.
        script_tag = soup.find('script', string=re.compile(r'"fund":{.*"canExchange"', re.DOTALL))
        
        if not script_tag:
            print(f"  [ERROR] Не удалось найти скрипт с данными JSON на странице {card_url}")
            return None

        # Используем re.DOTALL и здесь для надежности
        match = re.search(r'("fund":{.*?}),"cmsData":', script_tag.string, re.DOTALL)
        
        if not match:
            print(f"  [ERROR] Не удалось извлечь JSON из скрипта на странице {card_url}")
            if DEBUG:
                print(f"  [DEBUG] Содержимое script тега: {script_tag.string[:1000]}...")
            return None
            
        json_string = "{" + match.group(1) + "}"
        
        if DEBUG:
            print(f"  [DEBUG] Извлечен JSON (первые 300 символов): {json_string[:300]}...")

        data = json.loads(json_string)
        fund_json = data.get('fund', {})

        if not fund_json:
            print(f"  [ERROR] Объект 'fund' не найден в извлеченном JSON.")
            return None
        
        fund_data = {'url': card_url}

        # --- Извлечение данных из JSON ---
        fund_data['name'] = fund_json.get('shortName', 'N/A')
        fund_data['management_company'] = fund_json.get('fundCompNameRus', 'N/A')
        
        last_price_info = {}
        fund_prices = fund_json.get('fundPrice')
        if fund_prices and isinstance(fund_prices, list):
            last_price_info = fund_prices[-1]
        
        key_metrics = {
            "Стоимость пая": f"{last_price_info.get('dealPrice', 'N/A')} ₽",
            "Дата стоимости пая": last_price_info.get('priceDate', 'N/A'),
            "Активы фонда (СЧА)": f"{last_price_info.get('dealNetAssetValue', 'N/A')} ₽",
            "Минимальная инвестиция": f"{fund_json.get('minInvest', 'N/A')} ₽"
        }
        
        historical_yield = fund_json.get('historicalYield', {})
        for period, key in [('1 мес', 'historicalYieldMonth'), ('3 мес', 'historicalYieldQuarter'), ('6 мес', 'historicalYieldHalfYear'), ('1 год', 'historicalYieldYear')]:
            if historical_yield.get(key) is not None:
                key_metrics[f'Доходность, {period}'] = f"{historical_yield.get(key):.2f}%"

        fund_data['key_metrics'] = key_metrics
        
        strategy_html = fund_json.get('description', '')
        fund_data['strategy'] = BeautifulSoup(strategy_html, 'html.parser').get_text() if strategy_html else "N/A"

        fund_data['composition'] = fund_json.get('fundStructure', {})
        
        details = {
            "Управляющая компания": fund_json.get('fundCompNameRus', 'N/A'),
            "Номер регистрации": fund_json.get('registrationNumber', 'N/A'),
            "ISIN": fund_json.get('isin', 'N/A'),
            "Дата формирования": fund_json.get('registrationDate', 'N/A')
        }
        fund_data['details'] = details

        fund_data['commissions'] = fund_json.get('commissions', {})

        return fund_data

    except requests.exceptions.RequestException as e:
        print(f"  [ERROR] Не удалось загрузить карточку {card_url}: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"  [ERROR] Ошибка декодирования JSON с карточки {card_url}: {e}")
        return None
    except Exception as e:
        print(f"  [ERROR] Непредвиденная ошибка парсинга карточки {card_url}: {e}")
        return None


def main():
    """
    Основная функция для запуска всего процесса скрейпинга.
    """
    risk_urls = {
        "LOW": "https://finuslugi.ru/invest/funds?risk%5B0%5D=LOW&sortOrder=DESC&sortBy=YIELD_ALL&yieldSortField=YIELD_ALL",
        "MEDIUM": "https://finuslugi.ru/invest/funds?risk%5B0%5D=MEDIUM&sortOrder=DESC&sortBy=YIELD_ALL&yieldSortField=YIELD_ALL",
        "HIGH": "https://finuslugi.ru/invest/funds?risk%5B0%5D=HIGH&sortOrder=DESC&sortBy=YIELD_ALL&yieldSortField=YIELD_ALL"
    }
    
    all_funds = []

    for risk_level, url in risk_urls.items():
        print(f"\n{'='*15} Начинаю обработку уровня риска: {risk_level} {'='*15}")
        
        fund_links = get_fund_links(url)
        
        for link in fund_links:
            fund_details = parse_fund_card(link)
            if fund_details:
                fund_details['risk_level'] = risk_level
                all_funds.append(fund_details)
            time.sleep(1)
            
    print(f"\n\n{'='*20} СБОР ДАННЫХ ЗАВЕРШЕН {'='*20}")
    print(f"Всего собрано информации по {len(all_funds)} фондам.")
    
    print("\n--- РЕЗУЛЬТАТ ---")
    print(json.dumps(all_funds, ensure_ascii=False, indent=2))
    
    with open('all_funds_data.json', 'w', encoding='utf-8') as f:
        json.dump(all_funds, f, ensure_ascii=False, indent=2)
    print("\n[INFO] Данные также сохранены в файл all_funds_data.json")


if __name__ == "__main__":
    main()




=============== Начинаю обработку уровня риска: LOW ===============

[INFO] Загружаю список фондов: https://finuslugi.ru/invest/funds?risk%5B0%5D=LOW&sortOrder=DESC&sortBy=YIELD_ALL&yieldSortField=YIELD_ALL
[DEBUG] Статус-код ответа: 200
[DEBUG] Получен HTML (первые 300 символов): <!DOCTYPE html><html lang="ru"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1"/><link rel="stylesheet" href="/invest/funds/_next/static/css/c717c71e5a8da870.css" data-precedence="next"/><link rel="stylesheet" href="/invest/funds/_next/static/css/afce4...
[INFO] Найдено 7 уникальных ссылок на фонды.
  > Парсинг карточки: https://finuslugi.ru/invest/funds/713c2365-5080-4a21-be78-82fb4475e8bd
  [DEBUG] Статус-код ответа: 200
<!DOCTYPE html><html lang="ru"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1"/><link rel="stylesheet" href="/invest/funds/_next/static/css/c717c71e5a8da870.css" data-precedence="next"/><link r

KeyboardInterrupt: 